# Проверка Whisper


In [86]:
!pip install hiclass

In [87]:
class EmbeddingStorage:
    def __init__(self, labels=None, filenames=None, embeddings=None):
        """
        Initialize the EmbeddingStorage class.

        Args:
            labels (list or np.ndarray): An array of labels for the embeddings.
            filenames (list or np.ndarray): An array of filenames associated with the embeddings.
            embeddings (np.ndarray): A NumPy array containing all embeddings.
        """
        self.labels = np.array(labels) if labels is not None else np.array([])
        self.filenames = np.array(filenames) if filenames is not None else np.array([])
        self.embeddings = np.array(embeddings) if embeddings is not None else np.empty((0,))

    def add_embedding(self, label, filename, embedding):
        """
        Add a new embedding, along with its label and filename.

        Args:
            label (int or str): The label of the embedding.
            filename (str): The filename associated with the embedding.
            embedding (np.ndarray or torch.Tensor): The embedding to add (can be a NumPy array or Tensor).
        """
        if isinstance(embedding, np.ndarray):
            emb_array = embedding
        else:
            # Convert torch.Tensor to NumPy
            emb_array = embedding.cpu().numpy()

        # Append the new data
        self.labels = np.append(self.labels, label)
        self.filenames = np.append(self.filenames, filename)

        if self.embeddings.size == 0:
            self.embeddings = emb_array.reshape(1, -1)
        else:
            self.embeddings = np.vstack([self.embeddings, emb_array])

    def save_to_file(self, file_path):
        """
        Save the embeddings, labels, and filenames to a file (as .npz).

        Args:
            file_path (str): The path to save the .npz file.
        """
        np.savez(file_path, labels=self.labels, filenames=self.filenames, embeddings=self.embeddings)

    @classmethod
    def load_from_file(cls, file_path):
        """
        Load embeddings, labels, and filenames from a saved .npz file.

        Args:
            file_path (str): The path to the .npz file to load.

        Returns:
            EmbeddingStorage: An instance of EmbeddingStorage with loaded data.
        """
        data = np.load(file_path)
        return cls(labels=data['labels'], filenames=data['filenames'], embeddings=data['embeddings'])

    def get_embedding_by_filename(self, filename):
        """
        Retrieve an embedding by its associated filename.

        Args:
            filename (str): The filename to search for.

        Returns:
            np.ndarray: The corresponding embedding or None if not found.
        """
        if filename in self.filenames:
            idx = np.where(self.filenames == filename)[0][0]
            return self.embeddings[idx]
        else:
            return None


In [88]:
    def join_on_videoname(self, other_storage):
        """
        Join two EmbeddingStorage objects on the 'videoname' (filename). The embeddings will be stored as tuples.

        Args:
            other_storage (EmbeddingStorage): Another EmbeddingStorage object to join with.

        Returns:
            EmbeddingStorage: A new EmbeddingStorage object with merged data (embedding tuples).
        """
        # Find common filenames
        common_filenames = np.intersect1d(self.filenames, other_storage.filenames)

        # Initialize lists to store merged data
        merged_labels = []
        merged_filenames = []
        merged_embeddings = []

        for filename in common_filenames:
            # Get embeddings for the common filename from both storages
            idx_self = np.where(self.filenames == filename)[0][0]
            idx_other = np.where(other_storage.filenames == filename)[0][0]

            emb_self = self.embeddings[idx_self]
            emb_other = other_storage.embeddings[idx_other]

            # Store embeddings as a tuple
            merged_embedding = (emb_self, emb_other)

            # Get the label from the first storage (could be changed based on use case)
            merged_label = self.labels[idx_self]

            # Append to the merged data
            merged_labels.append(merged_label)
            merged_filenames.append(filename)
            merged_embeddings.append(merged_embedding)

        # Convert lists to numpy arrays
        merged_labels = np.array(merged_labels)
        merged_filenames = np.array(merged_filenames)
        merged_embeddings = np.array(merged_embeddings, dtype=object)

        # Return a new EmbeddingStorage instance with merged data
        return EmbeddingStorage(labels=merged_labels, filenames=merged_filenames, embeddings=merged_embeddings)

    def __len__(self):
        """
        Return the number of embeddings stored.
        """
        return len(self.labels)

    def __getitem__(self, idx):
        """
        Retrieve the label, filename, and embedding by index.

        Args:
            idx (int): The index of the embedding to retrieve.

        Returns:
            tuple: A tuple containing (label, filename, embedding).
        """
        if idx >= len(self.labels):
            raise IndexError("Index out of range")
        return self.labels[idx], self.filenames[idx], self.embeddings[idx]

    def __repr__(self):
        return f"EmbeddingStorage(labels={len(self.labels)}, filenames={len(self.filenames)}, embeddings_shape={self.embeddings.shape})"

In [89]:
import numpy as np

df = EmbeddingStorage.load_from_file('data/embeddings/whisper.npz')

In [90]:
import pandas as pd

train_data = pd.read_csv("train_data_categories.csv")
train_data.dropna()
train_data.head()

,video_id,title,description,tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира


In [91]:
import pandas as pd
from tqdm import tqdm

taxonomy = pd.read_csv('IAB_tags.csv')

def create_tags_to_labels(taxonomy):
    tags = []
    for i, row in tqdm(taxonomy.iterrows()):
        level1 = row['Уровень 1 (iab)'] if isinstance(row['Уровень 1 (iab)'], str) else ""
        level2 = row['Уровень 2 (iab)'] if isinstance(row['Уровень 2 (iab)'], str) else ""
        level3 = row['Уровень 3 (iab)'] if isinstance(row['Уровень 3 (iab)'], str) else ""
        tags.append([level1, level2, level3])
    return tags

# Extract the data into the required format
tags_to_labels = create_tags_to_labels(taxonomy)

# Optional: print the result or process further
print(" tags_to_labels: ",tags_to_labels)


611it [00:00, 4270.19it/s]

 tags_to_labels:  [['Транспорт', '', ''], ['Транспорт', 'Типы кузова автомобиля', ''], ['Транспорт', 'Типы кузова автомобиля', 'Грузовой автомобиль'], ['Транспорт', 'Типы кузова автомобиля', 'Седан'], ['Транспорт', 'Типы кузова автомобиля', 'Универсал'], ['Транспорт', 'Типы кузова автомобиля', 'SUV'], ['Транспорт', 'Типы кузова автомобиля', 'Фургон'], ['Транспорт', 'Типы кузова автомобиля', 'Кабриолет'], ['Транспорт', 'Типы кузова автомобиля', 'Купе'], ['Транспорт', 'Типы кузова автомобиля', 'Кроссовер'], ['Транспорт', 'Типы кузова автомобиля', 'Хэтчбек'], ['Транспорт', 'Типы кузова автомобиля', 'Микроавтомобиль'], ['Транспорт', 'Типы кузова автомобиля', 'Минивэн'], ['Транспорт', 'Типы кузова автомобиля', 'Внедорожник'], ['Транспорт', 'Типы кузова автомобиля', 'Пикап'], ['Транспорт', 'Типы автомобилей', ''], ['Транспорт', 'Типы автомобилей', 'Бюджетные автомобили'], ['Транспорт', 'Типы автомобилей', 'Подержанные автомобили'], ['Транспорт', 'Типы автомобилей', 'Классические автомобили']

In [92]:
from hiclass import LocalClassifierPerNode
from sklearn.ensemble import RandomForestClassifier

# Ensure that X_train and Y_train have the same number of samples
X_train = df.embeddings[:len(tags_to_labels)] # Use slicing to match the lengths
Y_train = tags_to_labels
X_test = df.embeddings[:len(tags_to_labels)] # Use slicing here as well

# Use random forest classifiers for every node
rf = RandomForestClassifier()
classifier = LocalClassifierPerNode(local_classifier=rf)

# Train local classifier per node
classifier.fit(X_train, Y_train)

# Predict
predictions = classifier.predict(X_test)
print("Predicted classes:", predictions)

Predicted classes: [['Транспорт' 'Типы автомобилей' 'Подержанные автомобили']
 ['Транспорт' 'Типы кузова автомобиля' 'Пикап']
 ['Транспорт' 'Типы кузова автомобиля' 'Грузовой автомобиль']
 ...
 ['Игры' 'Жанры видеоигр' 'Музыкальные видеоигры']
 ['Игры' 'Жанры видеоигр' 'Головоломки']
 ['Транспорт' 'Типы автомобилей' 'Беспилотные автомобили']]
